##Source : https://github.com/samcw/ResNet18-Pytorch

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )
            
    def forward(self, x):
        out = self.left(x)
        out = out + self.shortcut(x)
        out = F.relu(out)
        
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = self.make_layer(ResidualBlock, 64, 2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)        
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)        
        self.fc = nn.Linear(512, num_classes)
        
    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [8]:
def ResNet18():
    return ResNet(ResidualBlock)

In [9]:
#Use the ResNet18 on Cifar-10
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import argparse
import os

#check gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#set hyperparameter
EPOCH = 10
pre_epoch = 0
BATCH_SIZE = 128
LR = 0.01

#prepare dataset and preprocessing
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

#labels in CIFAR10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#define ResNet18
net = ResNet18().to(device)

#define loss funtion & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
#train
for epoch in range(pre_epoch, EPOCH):
    print('\nEpoch: %d' % (epoch + 1))
    net.train()
    sum_loss = 0.0
    correct = 0.0
    total = 0.0
    for i, data in enumerate(trainloader, 0):
        #prepare dataset
        length = len(trainloader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        #forward & backward
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #print ac & loss in each batch
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% ' 
              % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
        
    #get the ac with testdataset in each epoch
    print('Waiting Test...')
    with torch.no_grad():
        correct = 0
        total = 0
        for data in testloader:
            net.eval()
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
        print('Test\'s ac is: %.3f%%' % (100 * correct / total))

print('Train has finished, total epoch is %d' % EPOCH)


Epoch: 1
[epoch:1, iter:1] Loss: 2.404 | Acc: 7.031% 
[epoch:1, iter:2] Loss: 2.377 | Acc: 10.547% 
[epoch:1, iter:3] Loss: 2.344 | Acc: 11.458% 
[epoch:1, iter:4] Loss: 2.351 | Acc: 11.133% 
[epoch:1, iter:5] Loss: 2.346 | Acc: 11.875% 
[epoch:1, iter:6] Loss: 2.329 | Acc: 13.281% 
[epoch:1, iter:7] Loss: 2.318 | Acc: 14.174% 
[epoch:1, iter:8] Loss: 2.300 | Acc: 15.039% 
[epoch:1, iter:9] Loss: 2.283 | Acc: 16.059% 
[epoch:1, iter:10] Loss: 2.263 | Acc: 16.641% 
[epoch:1, iter:11] Loss: 2.242 | Acc: 17.330% 
[epoch:1, iter:12] Loss: 2.230 | Acc: 17.773% 
[epoch:1, iter:13] Loss: 2.225 | Acc: 18.269% 
[epoch:1, iter:14] Loss: 2.214 | Acc: 18.917% 
[epoch:1, iter:15] Loss: 2.206 | Acc: 19.167% 
[epoch:1, iter:16] Loss: 2.197 | Acc: 19.727% 
[epoch:1, iter:17] Loss: 2.187 | Acc: 20.221% 
[epoch:1, iter:18] Loss: 2.177 | Acc: 20.703% 
[epoch:1, iter:19] Loss: 2.161 | Acc: 21.094% 
[epoch:1, iter:20] Loss: 2.157 | Acc: 21.406% 
[epoch:1, iter:21] Loss: 2.152 | Acc: 21.838% 
[epoch:1, ite

[epoch:1, iter:174] Loss: 1.675 | Acc: 38.232% 
[epoch:1, iter:175] Loss: 1.673 | Acc: 38.299% 
[epoch:1, iter:176] Loss: 1.672 | Acc: 38.348% 
[epoch:1, iter:177] Loss: 1.670 | Acc: 38.422% 
[epoch:1, iter:178] Loss: 1.668 | Acc: 38.505% 
[epoch:1, iter:179] Loss: 1.666 | Acc: 38.582% 
[epoch:1, iter:180] Loss: 1.664 | Acc: 38.659% 
[epoch:1, iter:181] Loss: 1.662 | Acc: 38.747% 
[epoch:1, iter:182] Loss: 1.661 | Acc: 38.814% 
[epoch:1, iter:183] Loss: 1.659 | Acc: 38.879% 
[epoch:1, iter:184] Loss: 1.658 | Acc: 38.961% 
[epoch:1, iter:185] Loss: 1.656 | Acc: 39.003% 
[epoch:1, iter:186] Loss: 1.655 | Acc: 39.058% 
[epoch:1, iter:187] Loss: 1.653 | Acc: 39.146% 
[epoch:1, iter:188] Loss: 1.652 | Acc: 39.175% 
[epoch:1, iter:189] Loss: 1.651 | Acc: 39.199% 
[epoch:1, iter:190] Loss: 1.649 | Acc: 39.276% 
[epoch:1, iter:191] Loss: 1.649 | Acc: 39.287% 
[epoch:1, iter:192] Loss: 1.648 | Acc: 39.331% 
[epoch:1, iter:193] Loss: 1.647 | Acc: 39.366% 
[epoch:1, iter:194] Loss: 1.646 | Acc: 3

[epoch:1, iter:345] Loss: 1.468 | Acc: 46.261% 
[epoch:1, iter:346] Loss: 1.467 | Acc: 46.311% 
[epoch:1, iter:347] Loss: 1.466 | Acc: 46.344% 
[epoch:1, iter:348] Loss: 1.466 | Acc: 46.361% 
[epoch:1, iter:349] Loss: 1.465 | Acc: 46.376% 
[epoch:1, iter:350] Loss: 1.464 | Acc: 46.420% 
[epoch:1, iter:351] Loss: 1.463 | Acc: 46.439% 
[epoch:1, iter:352] Loss: 1.463 | Acc: 46.458% 
[epoch:1, iter:353] Loss: 1.462 | Acc: 46.516% 
[epoch:1, iter:354] Loss: 1.461 | Acc: 46.540% 
[epoch:1, iter:355] Loss: 1.460 | Acc: 46.547% 
[epoch:1, iter:356] Loss: 1.460 | Acc: 46.583% 
[epoch:1, iter:357] Loss: 1.459 | Acc: 46.612% 
[epoch:1, iter:358] Loss: 1.458 | Acc: 46.657% 
[epoch:1, iter:359] Loss: 1.456 | Acc: 46.718% 
[epoch:1, iter:360] Loss: 1.456 | Acc: 46.758% 
[epoch:1, iter:361] Loss: 1.455 | Acc: 46.801% 
[epoch:1, iter:362] Loss: 1.453 | Acc: 46.856% 
[epoch:1, iter:363] Loss: 1.452 | Acc: 46.907% 
[epoch:1, iter:364] Loss: 1.451 | Acc: 46.967% 
[epoch:1, iter:365] Loss: 1.450 | Acc: 4

..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


Test's ac is: 56.000%

Epoch: 2
[epoch:2, iter:392] Loss: 1.218 | Acc: 56.250% 
[epoch:2, iter:393] Loss: 1.150 | Acc: 60.156% 
[epoch:2, iter:394] Loss: 1.051 | Acc: 63.802% 
[epoch:2, iter:395] Loss: 1.119 | Acc: 60.547% 
[epoch:2, iter:396] Loss: 1.063 | Acc: 62.344% 
[epoch:2, iter:397] Loss: 1.075 | Acc: 61.849% 
[epoch:2, iter:398] Loss: 1.067 | Acc: 61.719% 
[epoch:2, iter:399] Loss: 1.039 | Acc: 63.281% 
[epoch:2, iter:400] Loss: 1.049 | Acc: 63.455% 
[epoch:2, iter:401] Loss: 1.043 | Acc: 64.219% 
[epoch:2, iter:402] Loss: 1.040 | Acc: 64.134% 
[epoch:2, iter:403] Loss: 1.059 | Acc: 63.542% 
[epoch:2, iter:404] Loss: 1.077 | Acc: 62.680% 
[epoch:2, iter:405] Loss: 1.081 | Acc: 62.277% 
[epoch:2, iter:406] Loss: 1.086 | Acc: 61.927% 
[epoch:2, iter:407] Loss: 1.093 | Acc: 61.377% 
[epoch:2, iter:408] Loss: 1.089 | Acc: 61.581% 
[epoch:2, iter:409] Loss: 1.085 | Acc: 61.849% 
[epoch:2, iter:410] Loss: 1.078 | Acc: 62.007% 
[epoch:2, iter:411] Loss: 1.074 | Acc: 62.070% 
[epoch:2

...

[epoch:8, iter:3085] Loss: 0.381 | Acc: 86.577% 
[epoch:8, iter:3086] Loss: 0.381 | Acc: 86.571% 
[epoch:8, iter:3087] Loss: 0.381 | Acc: 86.578% 
[epoch:8, iter:3088] Loss: 0.381 | Acc: 86.585% 
[epoch:8, iter:3089] Loss: 0.380 | Acc: 86.588% 
[epoch:8, iter:3090] Loss: 0.380 | Acc: 86.588% 
[epoch:8, iter:3091] Loss: 0.381 | Acc: 86.584% 
[epoch:8, iter:3092] Loss: 0.380 | Acc: 86.593% 
[epoch:8, iter:3093] Loss: 0.380 | Acc: 86.598% 
[epoch:8, iter:3094] Loss: 0.380 | Acc: 86.598% 
[epoch:8, iter:3095] Loss: 0.380 | Acc: 86.612% 
[epoch:8, iter:3096] Loss: 0.380 | Acc: 86.616% 
[epoch:8, iter:3097] Loss: 0.380 | Acc: 86.602% 
[epoch:8, iter:3098] Loss: 0.381 | Acc: 86.593% 
[epoch:8, iter:3099] Loss: 0.381 | Acc: 86.604% 
[epoch:8, iter:3100] Loss: 0.381 | Acc: 86.609% 
[epoch:8, iter:3101] Loss: 0.381 | Acc: 86.592% 
[epoch:8, iter:3102] Loss: 0.381 | Acc: 86.597% 
[epoch:8, iter:3103] Loss: 0.381 | Acc: 86.597% 
[epoch:8, iter:3104] Loss: 0.382 | Acc: 86.587% 
[epoch:8, iter:3105]

[epoch:9, iter:3252] Loss: 0.340 | Acc: 88.357% 
[epoch:9, iter:3253] Loss: 0.340 | Acc: 88.312% 
[epoch:9, iter:3254] Loss: 0.341 | Acc: 88.312% 
[epoch:9, iter:3255] Loss: 0.341 | Acc: 88.287% 
[epoch:9, iter:3256] Loss: 0.341 | Acc: 88.269% 
[epoch:9, iter:3257] Loss: 0.341 | Acc: 88.275% 
[epoch:9, iter:3258] Loss: 0.341 | Acc: 88.257% 
[epoch:9, iter:3259] Loss: 0.342 | Acc: 88.216% 
[epoch:9, iter:3260] Loss: 0.342 | Acc: 88.198% 
[epoch:9, iter:3261] Loss: 0.344 | Acc: 88.134% 
[epoch:9, iter:3262] Loss: 0.344 | Acc: 88.130% 
[epoch:9, iter:3263] Loss: 0.344 | Acc: 88.137% 
[epoch:9, iter:3264] Loss: 0.343 | Acc: 88.155% 
[epoch:9, iter:3265] Loss: 0.343 | Acc: 88.161% 
[epoch:9, iter:3266] Loss: 0.343 | Acc: 88.179% 
[epoch:9, iter:3267] Loss: 0.343 | Acc: 88.174% 
[epoch:9, iter:3268] Loss: 0.345 | Acc: 88.119% 
[epoch:9, iter:3269] Loss: 0.345 | Acc: 88.121% 
[epoch:9, iter:3270] Loss: 0.345 | Acc: 88.100% 
[epoch:9, iter:3271] Loss: 0.346 | Acc: 88.090% 
[epoch:9, iter:3272]

[epoch:9, iter:3420] Loss: 0.355 | Acc: 87.711% 
[epoch:9, iter:3421] Loss: 0.355 | Acc: 87.713% 
[epoch:9, iter:3422] Loss: 0.355 | Acc: 87.713% 
[epoch:9, iter:3423] Loss: 0.354 | Acc: 87.720% 
[epoch:9, iter:3424] Loss: 0.354 | Acc: 87.735% 
[epoch:9, iter:3425] Loss: 0.354 | Acc: 87.745% 
[epoch:9, iter:3426] Loss: 0.354 | Acc: 87.739% 
[epoch:9, iter:3427] Loss: 0.354 | Acc: 87.743% 
[epoch:9, iter:3428] Loss: 0.354 | Acc: 87.724% 
[epoch:9, iter:3429] Loss: 0.354 | Acc: 87.713% 
[epoch:9, iter:3430] Loss: 0.354 | Acc: 87.715% 
[epoch:9, iter:3431] Loss: 0.355 | Acc: 87.691% 
[epoch:9, iter:3432] Loss: 0.355 | Acc: 87.693% 
[epoch:9, iter:3433] Loss: 0.355 | Acc: 87.684% 
[epoch:9, iter:3434] Loss: 0.355 | Acc: 87.697% 
[epoch:9, iter:3435] Loss: 0.355 | Acc: 87.688% 
[epoch:9, iter:3436] Loss: 0.355 | Acc: 87.698% 
[epoch:9, iter:3437] Loss: 0.355 | Acc: 87.687% 
[epoch:9, iter:3438] Loss: 0.355 | Acc: 87.697% 
[epoch:9, iter:3439] Loss: 0.355 | Acc: 87.701% 
[epoch:9, iter:3440]

[epoch:10, iter:3585] Loss: 0.341 | Acc: 88.080% 
[epoch:10, iter:3586] Loss: 0.343 | Acc: 88.071% 
[epoch:10, iter:3587] Loss: 0.343 | Acc: 88.074% 
[epoch:10, iter:3588] Loss: 0.343 | Acc: 88.111% 
[epoch:10, iter:3589] Loss: 0.346 | Acc: 88.036% 
[epoch:10, iter:3590] Loss: 0.345 | Acc: 88.006% 
[epoch:10, iter:3591] Loss: 0.344 | Acc: 88.064% 
[epoch:10, iter:3592] Loss: 0.344 | Acc: 88.078% 
[epoch:10, iter:3593] Loss: 0.344 | Acc: 88.049% 
[epoch:10, iter:3594] Loss: 0.345 | Acc: 88.010% 
[epoch:10, iter:3595] Loss: 0.346 | Acc: 87.963% 
[epoch:10, iter:3596] Loss: 0.344 | Acc: 88.007% 
[epoch:10, iter:3597] Loss: 0.343 | Acc: 87.981% 
[epoch:10, iter:3598] Loss: 0.343 | Acc: 87.975% 
[epoch:10, iter:3599] Loss: 0.343 | Acc: 87.959% 
[epoch:10, iter:3600] Loss: 0.343 | Acc: 87.973% 
[epoch:10, iter:3601] Loss: 0.342 | Acc: 87.967% 
[epoch:10, iter:3602] Loss: 0.341 | Acc: 87.980% 
[epoch:10, iter:3603] Loss: 0.341 | Acc: 88.012% 
[epoch:10, iter:3604] Loss: 0.341 | Acc: 87.987% 


[epoch:10, iter:3749] Loss: 0.335 | Acc: 88.329% 
[epoch:10, iter:3750] Loss: 0.334 | Acc: 88.352% 
[epoch:10, iter:3751] Loss: 0.334 | Acc: 88.362% 
[epoch:10, iter:3752] Loss: 0.334 | Acc: 88.348% 
[epoch:10, iter:3753] Loss: 0.335 | Acc: 88.338% 
[epoch:10, iter:3754] Loss: 0.334 | Acc: 88.358% 
[epoch:10, iter:3755] Loss: 0.334 | Acc: 88.384% 
[epoch:10, iter:3756] Loss: 0.334 | Acc: 88.383% 
[epoch:10, iter:3757] Loss: 0.334 | Acc: 88.373% 
[epoch:10, iter:3758] Loss: 0.335 | Acc: 88.350% 
[epoch:10, iter:3759] Loss: 0.335 | Acc: 88.333% 
[epoch:10, iter:3760] Loss: 0.335 | Acc: 88.333% 
[epoch:10, iter:3761] Loss: 0.335 | Acc: 88.349% 
[epoch:10, iter:3762] Loss: 0.335 | Acc: 88.346% 
[epoch:10, iter:3763] Loss: 0.334 | Acc: 88.361% 
[epoch:10, iter:3764] Loss: 0.334 | Acc: 88.367% 
[epoch:10, iter:3765] Loss: 0.335 | Acc: 88.364% 
[epoch:10, iter:3766] Loss: 0.335 | Acc: 88.360% 
[epoch:10, iter:3767] Loss: 0.335 | Acc: 88.357% 
[epoch:10, iter:3768] Loss: 0.335 | Acc: 88.372% 
